#### Heatmapに関するコード
ファイルの名称について
- acと遺伝子の長さを対応させる
    - clinvar_ac2seqnum.json
- acと変異数を対応させる
    - clinvar_ac2varCount

In [ ]:
#ACと遺伝子の長さを対応させる
import json
from collections import defaultdict

Clinvar_ac2seqnum = defaultdict(list)
acs =[]
seq = ""

data = open("../uniprot_human.dat")
for line in data:
    if line.startswith("AC"):
        info = line.rstrip().split()
        for b in info[1:]:
            acs.append(b.replace(";",""))  
    if line.startswith("     "):
        seq += line.strip().replace(" ","")
    elif line.startswith("//"):
        Clinvar_ac2seqnum[acs[0]]=len(seq)
        acs = []
        seq = ""

with open('../clinvar_ac2seqnum.json', 'w') as f:
    json.dump(Clinvar_ac2seqnum, f, indent=4)

In [ ]:
#変異数カウント
import json
from collections import defaultdict

json_open = open("../clinvar_1ac.json","r")
Clinvar_1ac_human_classfied = json.load(json_open)

Clinvar_varCount = defaultdict(int)

for data in Clinvar_1ac_human_classfied:
    for data2 in Clinvar_1ac_human_classfied[data]:
        Clinvar_varCount[data]+=1
        
with open('../clinvar_ac2varCount.json', 'w') as f:
    json.dump(Clinvar_varCount, f, indent=4)

In [ ]:
#遺伝子の長さ、変異数で規格化
import json
import math
from collections import defaultdict

json_open = open('../ac2locs_classified_plural.json', 'r')
ac2subcellularlocations = json.load(json_open)

json_open = open('../clinvar_1ac.json', 'r')
Clivnar = json.load(json_open)

json_open = open('../clinvar_pd_normalized.json', 'r')
pd_Clinvar_classified = json.load(json_open)

json_open = open("../clinvar_ac2varCount.json","r")
Clinvar_varCount = json.load(json_open)


#subcel = ["All",'Mitochondrion', 'Others', 'Cytoplasm,Mitochondrion', 'Nucleus', 'Lysosome', 'Membrane', 'Secreted', 'Cytoplasm', 'Endoplasmic reticulum', 'Golgi', 'NoData', 'Cytoplasm,Nucleus']
subcel = ["All",'Mitochondrion', 'Others', 'Cytoplasm,Mitochondrion', 'Nucleus', 'Lysosome', 'Membrane', 'Secreted', 'Cytoplasm', "ER,Golgi",'NoData']


disease = {}
polymorphism = {}
for s in subcel:
    disease[s] = defaultdict(int)
    polymorphism[s] = defaultdict(int)

aa = ['Ile', 'Val', 'Leu', 'Phe', 'Cys', 'Met', 'Ala', 'Gly', 'Thr', 'Ser', 'Trp', 'Tyr', 'Pro', 'His', 'Glu', 'Gln', 'Asp', 'Asn', 'Lys', 'Arg']
counter = {}
for s in subcel:
    counter[s] = defaultdict(int)

numCounter = defaultdict(int)
otherslist = set()

for ac in Clivnar:
    #タンパク質の長さ
    gene_length = Clinvar_ac2seqnum[ac]
    #変異の数で規格化する
    varCount = Clinvar_varCount[ac]
    for data in Clivnar[ac]:
        if ac in ac2subcellularlocations.keys():
            for compartment in ac2subcellularlocations[ac]:
                numCounter[compartment] += 1
                #遺伝子の長さ考慮
                #counter[compartment][data["Variant"]["before"]+data["Variant"]["after"]] += 1/gene_length/varCount
                #考慮しない
                counter[compartment][data["Variant"]["before"]+data["Variant"]["after"]] += 1
                # 変異と疾患のカウント
                if data["Clinical_significance"]== "Disease":
                    #遺伝子の長さ考慮
                    #disease[compartment][data["Variant"]["before"]+data["Variant"]["after"]] += 1/gene_length/varCount
                    # 局在を考えずにカウント
                    #disease['All'][data["Variant"]["before"]+data["Variant"]["after"]] += 1/gene_length/varCount
                    #考慮しない
                    disease[compartment][data["Variant"]["before"]+data["Variant"]["after"]] += 1
                elif data["Clinical_significance"] == "Polymorphism":
                    #遺伝子の長さを考慮
                    #polymorphism[compartment][data["Variant"]["before"]+data["Variant"]["after"]] += 1/gene_length/varCount
                    # 局在を考えずにカウント
                    #polymorphism['All'][data["Variant"]["before"]+data["Variant"]["after"]] += 1/gene_length/varCount
                    #考慮しない
                    polymorphism[compartment][data["Variant"]["before"]+data["Variant"]["after"]] += 1

# 局在を考えずにカウントしたのだけで d/(d+p) を計算しておく
# これが局在を考えないときの All_pd になる
wo_localize = {}
for b in aa:
    wo_localize[b] = {}
    for a in aa:
        var = b + a
        d = 0 if var not in disease['All'] else disease['All'][var]
        p = 0 if var not in polymorphism['All'] else polymorphism['All'][var]
        if d + p != 0:
            wo_localize[b][a] = d/(d+p)
        else:
            wo_localize[b][a] = "-"


for s in subcel:
    print("{}\t{}".format(s,"\t".join(aa)))
    for b in aa:
        print("{}\t".format(b),end="")
        for a in aa:
            var = b + a
            d = 0 if var not in disease[s] else disease[s][var]
            p = 0 if var not in polymorphism[s] else polymorphism[s][var]

            if d + p != 0 and wo_localize[b][a] != '-':
                try:
                    try:
                        #常用対数とる
                        # print("{}\t".format(math.log((d / (d + p)) /(pd_Clinvar_classified["All"][b][a]))),end="")
                        # jsonから読み込んだ値ではなく上で計算した値を利用する
                        print("{}\t".format(math.log((d / (d + p)) /(wo_localize[b][a]))),end="")
                        #取らない
                        #print("{}\t".format((d / (d + p)),end=""))
                    except ValueError:
                        print("-\t",end="")
                except ZeroDivisionError:
                    print("-\t",end="")
            else:
                print("-\t",end="")
        print()
    print()

# ヒートマップをJSONにする
# heatmap = {}
# for s in subcel:
#     heatmap[s] = defaultdict(dict)
# for s in subcel:
#     for b in aa:
#         for a in aa:
#             var = b + a
#             d = 0 if var not in disease[s] else disease[s][var]
#             p = 0 if var not in polymorphism[s] else polymorphism[s][var]

#             if d + p != 0:
#                 try:
#                     #heatmap[s][b][a] = d / (d + p)
#                     try:
#                         heatmap[s][b][a] = math.log((d / (d + p)) /(pd_Clinvar_classified["All"][b][a]))
#                     except ValueError:
#                         heatmap[s][b][a] = "-"
#                 except ZeroDivisionError:
#                     heatmap[s][b][a] = "-"
#             else:
#                 heatmap[s][b][a] = "-"

#with open('pd_Clinvar_classified2_log.json', 'w') as f:
#     json.dump(heatmap, f, indent=4)
    
